In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def RandomForestLDA(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
# LDA Classifier Arousal=z
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, z_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, z_train)  
    z_predict = classifier.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
    #return f1 score and accurecies for valence and arousal
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

############### Happy And Sad ####################
def RandomForestHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

############### All Emotions ####################
def RandomForestCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=15)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=10, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]
    
def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        RandomForest=RandomForestLDA(UserName,Window,StartTime,EndTime)
        arousal+=[RandomForest[0]]
        valence+=[RandomForest[1]]
        F1_ScoreArousal+=[RandomForest[2]]
        F1_ScoreValence+=[RandomForest[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=RandomForestCombined(UserName,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=RandomForestHS(UserName,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for 31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))



        

s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 36, 40.0: 23, 80.0: 22, 20.0: 11, 100.0: 7, 0.0: 1})
Happy Sad
[57.6, 0.5672301587301587]
Array of average All Emotions accurecies for user s01
Counter({25.0: 26, 37.5: 25, 12.5: 21, 50.0: 15, 62.5: 7, 0.0: 6})
Combined Emotions
[30.375, 0.28735019841269843]
Arousal 54.125
Counter({50.0: 26, 62.5: 25, 75.0: 17, 37.5: 15, 25.0: 10, 87.5: 5, 12.5: 2})
Valence 46.625
Counter({50.0: 31, 37.5: 20, 25.0: 18, 62.5: 18, 75.0: 10, 12.5: 3})
Arousal Valence
[54.125, 46.625, 0.520705044955045, 0.4516088911088911]
s02
Array of average Happy Sad accurecies for user s02
Counter({60.0: 41, 80.0: 36, 100.0: 12, 40.0: 6, 20.0: 4, 0.0: 1})
Happy Sad
[68.6, 0.7022777777777778]
Array of average All Emotions accurecies for user s02
Counter({37.5: 32, 25.0: 21, 50.0: 21, 12.5: 10, 75.0: 8, 62.5: 6, 87.5: 1, 0.0: 1})
Combined Emotions
[39.625, 0.37032665945165943]
Arousal 58.75
Counter({62.5: 29, 50.0: 24, 75.0: 19, 37.5: 14, 87.5: 8, 25.0

Arousal 57.75
Counter({50.0: 30, 62.5: 28, 75.0: 16, 37.5: 11, 87.5: 9, 25.0: 4, 12.5: 2})
Valence 61.25
Counter({50.0: 29, 62.5: 28, 75.0: 25, 87.5: 7, 37.5: 7, 25.0: 3, 100.0: 1})
Arousal Valence
[57.75, 61.25, 0.5263119935619937, 0.6058838106338107]
s15
Array of average Happy Sad accurecies for user s15
Counter({50.0: 39, 75.0: 35, 25.0: 15, 100.0: 8, 0.0: 3})
Happy Sad
[57.5, 0.5511428571428572]
Array of average All Emotions accurecies for user s15
Counter({37.5: 32, 25.0: 25, 50.0: 20, 12.5: 16, 62.5: 3, 0.0: 3, 75.0: 1})
Combined Emotions
[32.875, 0.3059037698412698]
Arousal 51.125
Counter({62.5: 29, 50.0: 24, 37.5: 17, 25.0: 17, 75.0: 9, 87.5: 3, 100.0: 1})
Valence 58.375
Counter({50.0: 31, 62.5: 26, 75.0: 20, 37.5: 10, 25.0: 6, 87.5: 5, 100.0: 2})
Arousal Valence
[51.125, 58.375, 0.4798075258075258, 0.5812641802641803]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 37, 60.0: 23, 100.0: 21, 40.0: 13, 20.0: 5, 0.0: 1})
Happy Sad
[70.6, 0.6842777777777778]
A

Array of average Happy Sad accurecies for user s28
Counter({66.66666666666666: 43, 83.33333333333334: 24, 50.0: 18, 33.33333333333333: 9, 100.0: 6})
Happy Sad
[66.66666666666667, 0.667085016835017]
Array of average All Emotions accurecies for user s28
Counter({25.0: 32, 12.5: 26, 37.5: 18, 50.0: 15, 62.5: 6, 0.0: 3})
Combined Emotions
[29.25, 0.27913897907647905]
Arousal 58.875
Counter({50.0: 28, 75.0: 26, 62.5: 19, 37.5: 14, 87.5: 8, 25.0: 5})
Valence 68.375
Counter({75.0: 34, 62.5: 30, 87.5: 12, 50.0: 10, 100.0: 6, 37.5: 6, 12.5: 1, 25.0: 1})
Arousal Valence
[58.875, 68.375, 0.5795905205905205, 0.684436063936064]
s29
Array of average Happy Sad accurecies for user s29
Counter({83.33333333333334: 35, 66.66666666666666: 31, 100.0: 19, 50.0: 11, 33.33333333333333: 3, 16.666666666666664: 1})
Happy Sad
[75.5, 0.7510075757575757]
Array of average All Emotions accurecies for user s29
Counter({37.5: 25, 62.5: 24, 50.0: 23, 25.0: 11, 75.0: 9, 12.5: 8})
Combined Emotions
[46.375, 0.454506382506